# GANet Demo

The following code demo trains a neural network for GA (gradient artifacts) removal in EEG-fMRI (electroencephalography-functional MRI) dataset.
GANet is constructed based on Tensorflow platform. This work has not been published, so the complete code is not publicly accessible yet (it will be fully accessible after publication).

## The main principle
The true EEG component in raw EEG signals acquired in simutaneous EEG-fMRI is drowned in other artifacts show in **Fig. 1**, especially for high gradient artifacts(GA). 

 <img src="./Figs/EEG_signal_components.PNG" width = "750" height = "450" alt="Fig.1" align=center />

 GA comes from fast gradient magnetic field changes (for spacial localisation in MRI), which induces eddy current in EEG electrodes (**Fig.2**). Since the gradient magnetic field is artificial generated by gradient coils embedding in MRI scanner and differs from scanning sequences, we can deduce GA with relatively unchanged conduction matrix. However, the conduction matrix consisting of human tissues, air and other matrials, is too complex to perform numerical computation. So, I developed a neural network for solving $F$ in **Fig.2** to predict GA then remove it from raw signals.

  <img src="./Figs/GANet_principle.PNG" width = "800" height = "600" alt="Fig.1" align=center />

In [ ]:
# Import modules
import os
from pathlib import Path

from session import Session
from config import get_config

## Path setup

In [ ]:
# You can run this code on the computing cluster
workon_cluster = False
if workon_cluster:
    d_wk= Path('/gpfs/share/home/1801110251/jobs/20210825_da_lr/')
    d_data = Path('/gpfs/share/home/1801110251/projects/ganet/datasets/20210627/bv/')
else:
    d_wk= Path(os.getcwd() + '/GANet_data')
    d_data = Path('/disk1/jiayi/CMRServer148/project/GANet/data/20210717/eegbv')
    f_signal = '/disk1/jiayi/CMRServer148/project/GANet/data/clean_eeg/test.vhdr'

# Make directory storing results including the trained model and cleaned data.
d_op = d_wk / 'result'
d_op.mkdir(parents=True, exist_ok=True)

# Read configuration file
f_cfg = d_wk / 'config.yaml'
os.system('cp ' + str(f_cfg) + ' ' + str(d_op) + '/')
cfg = get_config(filename=f_cfg)
cfg.d_root = Path(os.getcwd())

d_log = d_op / 'training_logs'
d_model = d_op / 'trained_model' / 'non_cv_model'
d_output = d_op / 'cleaned_data' / 'non_cv_data'
d_pic = d_op / 'pic'
d_pic.mkdir(parents=True, exist_ok=True)

cfg.d_data = d_data
cfg.d_log = d_log
cfg.d_model = d_model
cfg.d_output = d_output
cfg.d_eval = None

print("Make directory completed!\n")
print('Current cfg file: \n' + str(f_cfg) + '\n')

## Initialize training session

In [ ]:
sub = '03'

d_pic = d_pic / sub
d_pic.mkdir(parents=True, exist_ok=True)

str_sub = sub
vec_idx_run = 1

s1 = Session('.vhdr', str_sub=str_sub, vec_idx_run=vec_idx_run, str_arch='gru_arch_017',
                random_seed=1997, verbose=2, overwrite=True, cv_mode=False, num_fold=5, cfg=cfg)

## Preparation for training

In [ ]:
# Read data through MNE
s1.load_all_dataset(f_signals=f_signal)

# Prepare dataset (train, validation and test) including dataset splitting, data augmentation and normalization
s1.prepare_training(training_all=True)
print("***Prepration completed!\n")

## Model training, cleaning raw data and saving cleaned data

In [ ]:
# Train the neural network
s1.train()
s1.plot_training_history(p_figure=d_pic)

# Predict GA and substract GA from raw signals
s1.clean()

# Save the neural network and cleaned signals
s1.save_model()
s1.save_databv()

## Evaluating the performance

In [ ]:
s1.evaluate(mode='test')
s1.plot_random_epoch(str_ch_exg='F3', mode='test', p_figure=d_pic)

  <img src="./Figs/Evaluation_results.PNG" width = "400" height = "500" alt="Fig.3" align=center />

  The following figure present one 5-s epoch. The orange line shows raw signals without true EEG information added by clean EEG (blue line) recorded outside of MRI scanner. The green line presents GANet-cleaned signals without pre-known clean EEG.
  
  <img src="./Figs/Signal.PNG" width = "800" height = "500" alt="Fig.4" align=center />